# Thực nghiệm thuật toán Random Forest trên tập dataset 3

<h2> Table of content </h2>
<div class ="alert alert-block alert-info" style="margin-top: 20px">
  <ol>
    <li><a href = "#1"> Tạo tập dữ liệu </a></li>
    <li><a href = "#2"> Thử nghiệm mô hình mặc định </a></li>
    <li><a href = "#tuning"> Tinh chỉnh siêu tham số  </a></li>
  </ol>
</div>

In [1]:
import pandas as pd

from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

In [2]:
data_train = pd.read_csv("Data3_train.csv")
data_test = pd.read_csv("Data3_test.csv")

data_train = data_train.drop(data_train.columns[0], axis = 1)
data_test = data_test.drop(data_test.columns[0], axis = 1)

data_train.head()

,order_time,is_acquisition,is_successful,delivery_arrangement,basket_amount_lc,delivery_fee_amount_lc,Payment Method,platform,vertical_class,vertical,is_affordable_freedelivery,is_affordable_item,is_affordable_gem,is_affordable_restaurant,is_affordable_voucher,is_affordable,City,ontime
0,90,0,0,0,155.00,14.99,0,2,0,3,0,0,0,0,0,0,2,0
1,46,0,1,1,228.00,0.00,2,0,0,3,1,0,0,0,0,1,2,0
2,30,0,0,0,15.00,9.99,0,0,0,3,0,0,0,0,0,0,2,0
3,118,0,1,0,30.00,22.49,1,2,0,3,0,0,0,0,0,0,2,0
4,27,0,1,0,44.35,0.00,1,2,1,4,1,0,0,0,0,1,4,0


Dữ liệu cần dự đoán ở đây là cột ontime

In [3]:
X_train = data_train.iloc[:, data_train.columns != "ontime"].values
y_train = data_train["ontime"].values

X_test = data_test.iloc[:, data_test.columns != "ontime"].values
y_test = data_test["ontime"].values

<h3 id = "2"> 2. Thử nghiệm mô hình mặc định </h3>

In [5]:
model = RandomForestClassifier()
model.fit(X_train, y_train)
predict = model.predict(X_test)
print(classification_report(y_test, predict))

              precision    recall  f1-score   support

           0       0.82      0.72      0.77     13268
           1       0.70      0.81      0.75     10709

    accuracy                           0.76     23977
   macro avg       0.76      0.76      0.76     23977
weighted avg       0.77      0.76      0.76     23977



<h3 id = "tuning"> 3. Tinh chỉnh siêu tham số </h3>

Các siêu tham số trong mô hình Random Forest bao gồm:
<li> <b> Number of estimators</b>: Đây là số lượng cây quyết định được sử dụng trong mô hình. Một số lượng cây quyết định quá ít có thể dẫn đến việc mô hình không đủ tốt, trong khi một số lượng quá nhiều có thể dẫn đến việc mô hình bị quá khớp.
<li> <b> Maximum depth </b>: Đây là độ sâu tối đa của mỗi cây quyết định. Một độ sâu cây quá lớn có thể dẫn đến việc mô hình bị quá khớp, trong khi một độ sâu quá thấp có thể dẫn đến việc mô hình không đủ tốt.
<li> <b> Min samples leaf </b>: Đây là số lượng mẫu tối thiểu cần thiết để tạo một nút lá trong cây quyết định. Một giá trị quá thấp có thể dẫn đến việc mô hình bị quá khớp, trong khi một giá trị quá cao có thể dẫn đến việc mô hình không đủ tốt.

Siêu tham số được tinh chỉnh bằng phương pháp đánh giá chéo (cross validaition) với việc tập dữ liệu được chia làm 10 phần không chồng chéo và có kích thước bằng nhau. 
Ngoài ra thước đo cho mô hình em chọn là F1 - score

In [6]:
from sklearn.model_selection import GridSearchCV

model = RandomForestClassifier()
param_grid = {
    'n_estimators': [60, 80, 100],
    'max_depth': [3, 4, 5],
    'min_samples_leaf': [1, 2, 3],
}

grid_search = GridSearchCV(model, param_grid, cv=10, scoring= 'f1_micro')
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)

{'max_depth': 5, 'min_samples_leaf': 3, 'n_estimators': 80}


In [4]:
model = RandomForestClassifier(n_estimators=80, max_depth=5, min_samples_leaf=3)
model.fit(X_train, y_train)
predict = model.predict(X_test)
print(classification_report(y_test, predict))

              precision    recall  f1-score   support

           0       0.99      0.62      0.77     13268
           1       0.68      1.00      0.81     10709

    accuracy                           0.79     23977
   macro avg       0.84      0.81      0.79     23977
weighted avg       0.85      0.79      0.79     23977



In [5]:
import os
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

if not os.path.exists('images'):
    os.makedirs('images')

for i in range(len(model.estimators_)):
    fig = plt.figure(figsize=(25,20))
    _ = plot_tree(model.estimators_[i], filled=True)
    fig.savefig(f"images/decistion_tree_{i+1}.png")
    plt.close()